In [6]:
import os
from LHEImport.LHEImport2 import read_lhe, tohdf5
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import vector
import mplhep as hep

In [7]:
df_run_01 = pd.read_hdf('event_files/hdf5/071221_eft.h5', 'run_01')

In [13]:
def ptot(particles, particle_pdgid):
    '''
    determine total transverse momentum of given particle, Z by default
    use with apply function for dataframes
    kwargs: - particles expects a pd dataframe column
            - particle_pdgid is the pdgid of the individual particle
    '''
    for p in particles: 
        if abs(p.pdgid) == particle_pdgid:
            return p.fourvec.pt

def event_weight(events):
    '''
    return weights from event objects when given column of events
    '''
    for event in events: 
        return event.weight

def eta(particles, particle_pdgid):
    '''
    determine eta of given particle
    use with apply function for dataframes
    kwargs: - particles expects a pd dataframe column
            - particle_pdgid is the pdgid of the individual particle
    '''
    for p in particles: 
        if abs(p.pdgid) == particle_pdgid:
            return p.fourvec.eta
        
def deltaphi(particles, pdgid1, pdgid2):
    '''
    determine the difference in phi between two given particles, identified by their pdgids
    kwargs: - particles: a pd dataframe column
            - pdgid1: particle 1
            - pdgid2: particle 2
    '''
    particle_list=[]
    for p in particles:
        if p.id==pdgid1 or p.id==pdgid2:
            particle_list.append(p)
    return particle_list[0].fourvec.deltaphi(particle_list[1].fourvec)

def deltaeta(particles, pdgid1, pdgid2):
    '''
    determine the difference in eta between two given particles, identified by their pdgids
    kwargs: - particles: a pd dataframe column
            - pdgid1: particle 1
            - pdgid2: particle 2
    '''
    particle_list=[]
    for p in particles:
        if p.id==pdgid1 or p.id==pdgid2:
            particle_list.append(p)
    return particle_list[0].fourvec.deltaeta(particle_list[1].fourvec)

def deltaR(particles, pdgid1, pdgid2):
    '''
    determine the difference in eta between two given particles, identified by their pdgids
    kwargs: - particles: a pd dataframe column
            - pdgid1: particle 1
            - pdgid2: particle 2
    '''
    particle_list=[]
    for p in particles:
        if p.id==pdgid1 or p.id==pdgid2:
            particle_list.append(p)
    return particle_list[0].fourvec.deltaR(particle_list[1].fourvec)

def listparticles(particles): 
    '''
    takes the first row of a dataframe and outputs an array of pdgids for _all_ involved particles
    has it's flaws but often useful for sanity checks
    '''
    all_pdgids = []
    for particle in particles[0]:
        if particle.pdgid not in all_pdgids:
            all_pdgids.append(particle.pdgid)
    return all_pdgids

# def deltaR(particles, pdgid1, pdgid2):
#     '''
#     deltaR = sqrt(deltaeta^2 + deltaphi^2)
#     kwargs: - particles: pd.dataframe col of particle objects
#             - pdigid1: particle 1 
#             - pdgid2: particle 2 
      ### built this but found deltaR function in vector package ###
#     '''
    
#     delta_phi = deltaphi(particles, pdgid1, pdgid2)
#     delta_eta = deltaeta(particles, pdgid1, pdgid2)
#     return np.sqrt(delta_phi**2 + delta_eta**2)

# def jet(particles):
#     '''
#     jet is defined in our madgraph settings as: 
#     define j = g u c d s u~ c~ d~ s~
#     so I've assigned the corresponding pdgids to an array in this function
    
#     this function will check for all particles that match these pdgids and are also decayed particles
#     "decayed particles" are defined by their status
    
#     kwargs: - particles: pd.dataframe col of particle objects
    
#     returns: combined fourvector object of all constituents of jet

#     '''
#     jet_pdgids = [1,2,3,4,5,9,21]
#     particle_list = []
#     particle = vector.obj(pt=0, eta=0, phi=0, E=0)
#     for p in particles: 
#         if p.pdgid in jet_pdgids and p.status == 1 :
#             particle+=p.fourvec
#     return particle

def particlebypdgid(particles, pdgid):
    '''
    given a list of particles and a single pdgid, the vector object of the particle will be returned
    
    '''
    for p in particles: 
        if p.pdgid == pdgid:
            return p.fourvec

        
def cosstarzlep(particles):
    '''
    the cosine of the angle
    between the direction of the Z boson in the detector reference 
    frame, and the direction of the negatively-charged lepton from
    the Z boson decay in the rest frame of the Z boson
    
    to do this we need the Z fourvec
    identify -ve lepton (+ pdgid bc leptons are -ve)
    apply boost_p4(four_vector): change coordinate system 
    using another 4D vector as the difference
    typically apply the negative 4 vec?
    '''
    
    for p in particles: 
        vecs=[]
        if p.pdgid == 23:
            z = p.fourvec
        elif p.pdgid == 13: 
            mu_p = p.fourvec
            
    mu_p_boost = mu_p.boost_p4(z)
    return np.cos(z.deltaangle(mu_p_boost))
    
            
            
        


In [14]:
# z's
##extracting pt(Z)
df_run_01['pt_z'] = df_run_01.apply(lambda r: ptot(r['particles'],23), axis=1)
## extracting eta(Z)
df_run_01['eta_z'] = df_run_01.apply(lambda r: eta(r['particles'],23), axis=1)
## calc delta phi from two leptons from the Z, in this case the mu+ and mu-
df_run_01['deltaphi_ll_Z'] = df_run_01.apply(lambda r: deltaphi(r['particles'], 13, -13), axis=1)

# t's
## extracting pt(t)
df_run_01['pt_t'] = df_run_01.apply(lambda r: ptot(r['particles'],6 ), axis=1)
## extracting eta(t)
df_run_01['eta_t'] = df_run_01.apply(lambda r: eta(r['particles'],6), axis=1)

# t~'s
## extracting pt(t~)
df_run_01['pt_tbar'] = df_run_01.apply(lambda r: ptot(r['particles'],-6 ), axis=1)
## extracting eta(t~)
df_run_01['eta_tbar'] = df_run_01.apply(lambda r: eta(r['particles'],-6), axis=1)

# deltaR
df_run_01['dR_t_z'] = df_run_01.apply(lambda r: deltaR(r['particles'], 6, 23), axis=1)

# cos theta star z
df_run_01['cosstar'] = df_run_01.apply(lambda r: cosstarzlep(r['particles']), axis=1)

In [47]:
df_run_01.head()

,event_info,particles,weights,pt_z,eta_z,deltaphi_ll_Z,pt_t,eta_t,pt_tbar,eta_tbar,dR_t_z,cosstar
0,<LHEImport.LHEImport2.LHEEventInfo object at 0...,"[Particle, PDGID21, Particle, PDGID21, Particl...",{},228.775431,-0.338773,-0.988121,413.397520,-0.937775,None,None,1.242560,0.999523
1,<LHEImport.LHEImport2.LHEEventInfo object at 0...,"[Particle, PDGID21, Particle, PDGID21, Particl...",{},115.134232,0.153147,-1.268970,82.651300,-2.597080,None,None,3.282276,0.977079
2,<LHEImport.LHEImport2.LHEEventInfo object at 0...,"[Particle, PDGID21, Particle, PDGID21, Particl...",{},310.190923,0.166703,-0.511280,292.784693,0.380356,None,None,3.077590,0.999976
3,<LHEImport.LHEImport2.LHEEventInfo object at 0...,"[Particle, PDGID21, Particle, PDGID21, Particl...",{},275.477583,-0.005348,0.577442,287.954335,0.630422,None,None,1.938562,0.998339
4,<LHEImport.LHEImport2.LHEEventInfo object at 0...,"[Particle, PDGID21, Particle, PDGID21, Particl...",{},44.939786,2.480726,-2.205378,228.821558,0.223751,None,None,2.285043,0.997790


In [16]:
np.mean(df_run_01['cosstar'])

0.9493760337850644

In [46]:
# some testing to see if the angles make sense
angles = []
for i in np.arange(0,100):
    for p in df_run_01['particles'][i]:
        if p.pdgid == 13:
            mu = p.fourvec
        elif p.pdgid == 23:
            z = p.fourvec

    # print(mu)
    # print(z)

    mu_boost = mu.boost_p4(z)
    # print(mu_boost)

    # print('angle between normally', z.deltaangle(mu))
    # print('angle between boosted', z.deltaangle(mu_boost))
    # angles.append((z.deltaangle(mu_boost)))
    
    angles.append(np.cos(z.deltaangle(mu_boost)))
print(angles)

[0.9995234795111241, 0.9770791188966464, 0.9999761988485738, 0.9983390942317732, 0.9977902909829488, 0.9994234104387221, 0.9997461228751064, 0.9998051178635599, 0.9998197575055195, 0.9151259574158959, 0.999529455681227, 0.9990892122351033, 0.9654474440234542, 0.9985904099383037, 0.9999267101733776, 0.9836192642171167, 0.9920294077067673, 0.999981343761738, 0.9602874580886801, 0.9979003564767325, 0.9999917538566674, 0.9999913773007375, 0.9950358437147165, 0.999958065569777, 0.999996287548628, 0.9998254820071197, 0.9990388967449572, 0.7950115857092971, 0.4431643228555291, 0.9996318109865772, 0.9999543872382052, 0.9833580280934202, 0.9999699968070218, 0.9863376275706441, 0.9999997186241484, 0.9927394243696959, 0.9997477644666896, 0.9988876310759177, 0.9998277423905138, 0.9999859398506521, 0.5727153747695108, 0.9996579872847656, 0.9969443727116747, 0.9998248009451247, 0.9976125791744204, 0.9784669696166526, 0.9992454658318781, -0.10700119447482437, 0.9978436224972438, 0.9999352308679179, 0